<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Solving Tasks Requiring Web Info

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about the stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [1]:
import os
config_list = [
    {"api_type": "groq", "model": "llama-3.1-70b-versatile", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-70b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-70b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "mixtral-8x7b-32768", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-8b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "gemma2-9b-it", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-3.1-8b-instant", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-guard-3-8b", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-8b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    ]

llm_config={"config_list" : config_list}

In [2]:
# %pip install "pyautogen>=0.2.3" docker

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [3]:
import autogen

'''config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)'''

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-01',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-01',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [4]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [5]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

user_proxy (to assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
assistant (to user_proxy):

To determine who should read this paper, I need to collect some information about the paper. I will use Python to fetch the paper's title, abstract, and keywords.

```python
# filename: paper_info.py
import requests
from bs4 import BeautifulSoup

def get_paper_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1', class_='title').text.strip()
    abstract = soup.find('blockquote', class_='abstract').text.strip()
    keywords = [span.text.strip() for span in soup.find_all('span', class_='subject')]
    return title, abstract, keywords

url = 'https://arxiv.org/abs/2308.08155'
title, abstract, keywords = get_paper_info(url)
print('Title:', title)
print('Abstract:')
print(abstract)
print('Keywords:', keywords)
```

Pleas

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Title: Title:AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Abstract:
Abstract:AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-mak

ChatResult(chat_id=None, chat_history=[{'content': '\nWho should read this paper: https://arxiv.org/abs/2308.08155\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "To determine who should read this paper, I need to collect some information about the paper. I will use Python to fetch the paper's title, abstract, and keywords.\n\n```python\n# filename: paper_info.py\nimport requests\nfrom bs4 import BeautifulSoup\n\ndef get_paper_info(url):\n    response = requests.get(url)\n    soup = BeautifulSoup(response.text, 'html.parser')\n    title = soup.find('h1', class_='title').text.strip()\n    abstract = soup.find('blockquote', class_='abstract').text.strip()\n    keywords = [span.text.strip() for span in soup.find_all('span', class_='subject')]\n    return title, abstract, keywords\n\nurl = 'https://arxiv.org/abs/2308.08155'\ntitle, abstract, keywords = get_paper_info(url)\nprint('Title:', title)\nprint('Abstract:')\nprint(abstract)\nprint('Keywords:', keywords)\n```\n\nPlease 

## Example Task: Chat about Stock Market

In [7]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="""Show me the YTD gain of 10 largest technology companies as of today.""",
)

user_proxy (to assistant):

Show me the YTD gain of 10 largest technology companies as of today.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To find the YTD gain of the 10 largest technology companies, we need to first identify these companies and then fetch their current stock prices and YTD gains. 

Here's a plan to solve this task:

1. Identify the 10 largest technology companies.
2. Fetch their current stock prices and YTD gains.

To identify the 10 largest technology companies, we can use the market capitalization data from a reliable source like Wikipedia. However, to fetch their current stock prices and YTD gains, we can use a Python script that uses the Yahoo Finance API.

```python
# filename: tech_companies_ytd_gain.py
import yfinance as yf
import pandas as pd

# List of 10 largest technology companies
tech_companies = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC']

# Create a d

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)



>>>>>>>> NO HUMAN INPUT RECEIVED.


In [11]:
print(chat_result.summary)  # print the summary of the chat_result

The code has successfully fetched the YTD gains for the 10 largest technology companies. Here are the results:

1. Apple (AAPL): 29.79%
2. Microsoft (MSFT): 36.17%
3. Alphabet (GOOGL): 24.09%
4. Amazon (AMZN): 47.76%
5. Meta (META): 87.82%
6. Tesla (TSLA): 2.95%
7. NVIDIA (NVDA): 186.36%
8. PayPal (PYPL): 33.10%
9. Adobe (ADBE): 2.44%
10. Intel (INTC): -32.37%

Note that the YTD gains are calculated based on the historical stock price data from the beginning of the current year. The results show that Meta and NVIDIA have had the highest YTD gains, while Intel has had a significant loss.

To make the output more readable, we can format the YTD gains as percentages with two decimal places. Here's the updated code:

```python
# filename: tech_companies_ytd_gain.py
import yfinance as yf
import pandas as pd
from datetime import datetime

# List of 10 largest technology companies
tech_companies = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC']

# Create a di